In [1]:
!pip install scikit-learn==0.23.2
!pip install pycaret[full]
from pycaret.classification import *

In [3]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [97]:
train = pd.read_csv('/content/drive/MyDrive/MLIA/Data/titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/MLIA/Data/titanic/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/MLIA/Data/titanic/gender_submission.csv')
all_data = pd.concat([train, test], axis=0)

In [98]:
all_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)

In [99]:

all_data['Age'].fillna('24', inplace=True)  #use mode to fillna(age)
all_data['Fare'].fillna(all_data['Fare'].mean(), inplace=True)
all_data['Embarked'].fillna('S', inplace=True)


In [100]:
std = StandardScaler()
all_data['Pclass'] = std.fit_transform(np.array(all_data['Pclass']).reshape(-1,1))
all_data['Age'] = std.fit_transform(np.array(all_data['Age']).reshape(-1,1))
all_data['SibSp'] = std.fit_transform(np.array(all_data['SibSp']).reshape(-1,1))
all_data['Parch'] = std.fit_transform(np.array(all_data['Parch']).reshape(-1,1))
all_data['Fare'] = std.fit_transform(np.array(all_data['Fare']).reshape(-1,1))

In [101]:
all_data = pd.get_dummies(all_data, drop_first=True)
print(all_data.head())

   Survived    Pclass       Age     SibSp  Parch      Fare  Sex_male  \
0       0.0  0.841916 -0.511722  0.481288 -0.445 -0.503595         1   
1       1.0 -1.546098  0.710389  0.481288 -0.445  0.734503         0   
2       1.0  0.841916 -0.206195 -0.479087 -0.445 -0.490544         0   
3       1.0 -1.546098  0.481243  0.481288 -0.445  0.382925         0   
4       0.0  0.841916  0.481243 -0.479087 -0.445 -0.488127         1   

   Embarked_Q  Embarked_S  
0           0           1  
1           0           0  
2           0           1  
3           0           1  
4           0           1  


In [102]:
train = all_data.iloc[:891, :]
test = all_data.iloc[891:, :]
print(test.shape)

(418, 9)


In [45]:
s = setup(data=train,
          session_id=1,
          target='Survived',
          train_size=0.99,
          fold_strategy='stratifiedkfold',
          fold=5,
          fold_shuffle=True,
          silent=True,
          ignore_low_variance=True,
          remove_multicollinearity = True,
          normalize = True,
          normalize_method = 'robust')

,Description,Value
0,session_id,1
1,Target,Survived
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(891, 9)"
5,Missing Values,False
6,Numeric Features,8
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='Survived',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_st...
                ('dummy', Dummify(target='Survived')),
                ('fix_perfect', Remove_100(target='Survived')),
                ('clean_names', Clean_Colum_Names()),
 

In [46]:
model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8356,0.8689,0.7128,0.8366,0.7680,0.6421,0.6487,1.284
gbc,Gradient Boosting Classifier,0.8310,0.8709,0.7099,0.8245,0.7627,0.6327,0.6371,0.114
xgboost,Extreme Gradient Boosting,0.8276,0.8683,0.7514,0.7919,0.7695,0.6321,0.6343,0.414
lightgbm,Light Gradient Boosting Machine,0.8265,0.8752,0.7335,0.8001,0.7630,0.6269,0.6305,0.208
rf,Random Forest Classifier,0.8174,0.8670,0.7426,0.7740,0.7565,0.6107,0.6125,0.580
ada,Ada Boost Classifier,0.8129,0.8482,0.7427,0.7660,0.7522,0.6023,0.6044,0.112
et,Extra Trees Classifier,0.8072,0.8427,0.7277,0.7619,0.7433,0.5891,0.5906,0.546
qda,Quadratic Discriminant Analysis,0.7993,0.8457,0.7159,0.7529,0.7318,0.5719,0.5744,0.024
lr,Logistic Regression,0.7959,0.8513,0.7131,0.7466,0.7279,0.5650,0.5668,1.682
knn,K Neighbors Classifier,0.7959,0.8398,0.7218,0.7413,0.7308,0.5665,0.5673,0.144


INFO:logs:create_model_container: 16
INFO:logs:master_model_container: 16
INFO:logs:display_container: 2
INFO:logs:<catboost.core.CatBoostClassifier object at 0x7f00d23a1f90>
INFO:logs:compare_models() succesfully completed......................................


In [ ]:
catboost_best = create_model('catboost')

df_pred = predict_model(catboost_best, test)
y_pred = df_pred.loc[:, ['Label']]
y_pred = y_pred.astype(int)

In [ ]:
submission['Survived'] = y_pred
submission['Survived'] = submission['Survived'].astype(int)
submission.to_csv('/content/drive/MyDrive/MLIA/Data/titanic/submission.csv', index=False)
print(submission['Survived'].dtype)